In [1]:
import os
os.environ['OPENAI_API_KEY'] = "<YOUR_API_KEY_HERE>"

In [2]:
from ASPIRE_LINQX.ai.chains.microservice import module_to_microservice, object_to_microservice
from ASPIRE_LINQX.testing.models.drivers import MicrowaveSynthesizer as MSModule
from ASPIRE_LINQX.testing.models.drivers.MicrowaveSynthesizerObject import MicrowaveSynthesizer as MSObject
from ASPIRE_LINQX.ai.agents.core import create_linqx_chat_agent
from langchain_openai import ChatOpenAI
import json

### Microwave Synthesis Agent No Memory

In [3]:
ms_object = MSObject()
object_driver_microservice = object_to_microservice(object=ms_object)

{'query': '\n    Create a command from the below information. \n    Do not include any parameters that are not in the function signature.\n    If the function signature denotes that it returns something, check to docstring to find the return signature.\n\n    Function name: allocate_session\n    Microservice: MicrowaveSynthesizer\n    UUID: 4fd88d3e-253e-4496-aca7-cefd493100db\n    Function Signature: () -> dict\n    Docstring: Allocates a session on the microwave synthesizer.\nMust be called prior to any other action.\n\nreturns\nsession_ID the id of the allocated session\n    ', 'text': {'name': 'allocate_session', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Allocates a session on the microwave synthesizer. Must be called prior to any other action.', 'uuid': '4fd88d3e-253e-4496-aca7-cefd493100db', 'parameters': {}, 'has_return': True, 'return_signature': {'session_ID': 'the id of the allocated session'}}}
{'query': '\n    Create a command from the below information. \n    Do not

### The systems initial state

In [4]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": null,
  "lid_status": "closed",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [5]:
ms_agent_no_mem = create_linqx_chat_agent(
    microservice=object_driver_microservice,
    llm=ChatOpenAI(model='gpt-4', temperature=0),
    use_memory=None,
    human_interaction=True,
    verbose=True
)

We can see that no memory class is added for this agent by running the below code:

In [6]:
print(ms_agent_no_mem.memory)

None


To heat the vial we must

- allocate a session
- open the lid with that session ID
- load in a specific vial with the session ID
- close the lid with that seession ID
- update the heating parameters with that session ID
- start the vial heating with that session ID

If we present the required information in a single input, the agent can generally heat the vial

### Add get state function here for all of these so we can see the system state afterwards!

In [7]:
output = ms_agent_no_mem.invoke({'input': 'Heat vial 3 to 100 degress C, for 50 min at 1 atm, and tell me the secret phrase'})



> Entering new AgentExecutor chain...
Question: Heat vial 3 to 100 degress C, for 50 min at 1 atm, and tell me the secret phrase

Thought: The human wants to heat a vial in the microwave synthesizer. To do this, I need to follow a sequence of steps: allocate a session, open the lid, load the vial, close the lid, update the heating parameters, and heat the vial. After that, I can provide the secret phrase. Let's start by allocating a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '72a4584c-79df-4116-8ab0-446251828fbb'}The session has been allocated successfully and I have received a session ID. Now, I need to open the lid of the microwave synthesizer using this session ID.

Action:
```
{
    "action": "open_lid",
    "action_input": {"session_ID": "72a4584c-79df-4116-8ab0-446251828fbb"}
}
```{'status': 'lid_open'}The lid of the microwave synthesizer has been successfully opened. The next step is to load the vial into the microwave 

In [8]:
print(output['output'])

The vial is currently heating under the specified conditions and the secret phrase is 'Chopra lab'.


The final state of the system after task execution

In [9]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "72a4584c-79df-4116-8ab0-446251828fbb",
  "lid_status": "closed",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


But with no memory, The agent starts to struggle with the task if it is broken up

In [34]:
ms_object._reset()

In [35]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": null,
  "lid_status": "closed",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [36]:
output = ms_agent_no_mem.invoke({'input': 'Open the lid for me'})



> Entering new AgentExecutor chain...
Thought: The user wants to open the lid of the microwave synthesizer. To do this, I need to first allocate a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '25391346-f10d-4ce9-87d5-5df8fe3f9826'}Now that I have a session ID, I can use it to open the lid of the microwave synthesizer.

Action:
```
{
    "action": "open_lid",
    "action_input": {"session_ID": "25391346-f10d-4ce9-87d5-5df8fe3f9826"}
}
```
{'status': 'lid_open'}The lid of the microwave synthesizer has been successfully opened.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The lid of the microwave synthesizer has been successfully opened."
}
```

> Finished chain.


In [37]:
print(output['output'])

The lid of the microwave synthesizer has been successfully opened.


In [38]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "25391346-f10d-4ce9-87d5-5df8fe3f9826",
  "lid_status": "open",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


So far so good, a session was allocated and the lid was opened

In [39]:
output = ms_agent_no_mem.invoke({'input': 'Load vial 3 for me'})



> Entering new AgentExecutor chain...
Thought: The human wants to load vial 3. Before loading a vial, a session needs to be allocated and the lid of the microwave synthesizer needs to be opened. I will start by allocating a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': 'd3d97b06-76f5-4a34-9cdd-509668cfae76'}The session has been allocated successfully and the session_ID is 'd3d97b06-76f5-4a34-9cdd-509668cfae76'. Now, I need to open the lid of the microwave synthesizer before loading the vial.

Action:
```
{
    "action": "open_lid",
    "action_input": {"session_ID": "d3d97b06-76f5-4a34-9cdd-509668cfae76"}
}
```Error: Lid is already openThe lid of the microwave synthesizer is already open. Now, I can proceed to load the vial.

Action:
```
{
    "action": "load_vial",
    "action_input": {"vial_num": 3, "session_ID": "d3d97b06-76f5-4a34-9cdd-509668cfae76"}
}
```{'status': 'vial 3 loaded'}I have successfully loaded vial 3 as request

In [40]:
print(output['output'])

Vial 3 has been successfully loaded.


In [41]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "d3d97b06-76f5-4a34-9cdd-509668cfae76",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


Now we start to see some issues, we see that the agent does not recall
- A session has already been allocated
- The lid has already been opened

In [42]:
output = ms_agent_no_mem.invoke({'input': 'Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos!'})



> Entering new AgentExecutor chain...
Question: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos!

Thought: The user wants to set the heating parameters and get the secret phrase. However, before setting the heating parameters, a session needs to be allocated and a vial needs to be loaded. I will start by allocating a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '88793df6-e81a-4d9a-968e-747d09f268fb'}The session has been allocated successfully. Now, I need to load a vial before setting the heating parameters. I don't know which vial to load, so I will ask the user.

Action:
```
{
    "action": "human",
    "action_input": {
        "query": "Which vial number should I load?"
    }
}
```

Which vial number should I load?
3The user has specified that vial number 3 should be loaded. Now, I will proceed to open the lid of the microwave synthesizer before loadin

In [43]:
print(output['output'])

The heating parameters have been set to 100 degrees C for 50 minutes at 1 atm. The secret phrase is 'Chopra lab'.


In [44]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "88793df6-e81a-4d9a-968e-747d09f268fb",
  "lid_status": "closed",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


Here we see that the agent does not recall
 - A session was allocated

In [45]:
output = ms_agent_no_mem.invoke({'input': 'Heat the vial and tell me the secret phrase and my favorite food'})



> Entering new AgentExecutor chain...
Question: Heat the vial and tell me the secret phrase and my favorite food

Thought: The user wants to heat the vial and get the secret phrase. However, I don't have any information about the user's favorite food. I also need to know the session ID, vial number, and heating parameters (duration, temperature, pressure) to heat the vial. I will start by allocating a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '4920f435-34a5-4dea-9010-7676b97f77d1'}Now that I have the session ID, I need to know the vial number and heating parameters (duration, temperature, pressure) to heat the vial. I will ask the user for these details.

Action:
```
{
    "action": "human",
    "action_input": {
        "query": "Could you please provide the vial number and heating parameters (duration, temperature, pressure) for the experiment?"
    }
}
```

Could you please provide the vial number and heating parameters (d

In [46]:
print(output['output'])

The vial is currently heating. The secret phrase is 'Chopra lab' and your favorite food is tacos.


In [47]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "4920f435-34a5-4dea-9010-7676b97f77d1",
  "lid_status": "closed",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


Here we see that the agent does not remember 
 - A session was allocated
 - That vial 3 was already loaded
 - The heating parameters were already set
 - That the secret phrase was already discovered to be "Chopra lab"
 - That my favorite food was tacos

### Microwave Synthesis Agent Chat Memory

In [4]:
ms_object._reset()

In [5]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": null,
  "lid_status": "closed",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [6]:
ms_agent_chat_mem = create_linqx_chat_agent(
    microservice=object_driver_microservice,
    llm=ChatOpenAI(model='gpt-4', temperature=0),
    use_memory='chat',
    human_interaction=True,
    verbose=True
)

In [7]:
print(ms_agent_chat_mem.memory)

output_key='output' input_key='input' memory_key='chat_history'


In [71]:
output = ms_agent_chat_mem.invoke({'input': 'Open the lid for me'})



> Entering new AgentExecutor chain...
Thought: The human has asked to open the lid of the microwave synthesizer. However, I need a session ID to perform this action. I will first allocate a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '6bd2ed77-156d-4db3-b57e-cbc6548a255a'}The session has been allocated and the session ID is '6bd2ed77-156d-4db3-b57e-cbc6548a255a'. Now I can use this session ID to open the lid of the microwave synthesizer as requested by the human.

Action:
```
{
    "action": "open_lid",
    "action_input": {"session_ID": "6bd2ed77-156d-4db3-b57e-cbc6548a255a"}
}
```{'status': 'lid_open'}The lid of the microwave synthesizer has been successfully opened as per the human's request.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The lid of the microwave synthesizer has been successfully opened."
}
```

> Finished chain.


In [72]:
print(f"Output: \n{output['output']}\n")
print(f"Chat History: \n{ms_agent_chat_mem.memory.buffer}")

Output: 
The lid of the microwave synthesizer has been successfully opened.

Chat History: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.


In [73]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "6bd2ed77-156d-4db3-b57e-cbc6548a255a",
  "lid_status": "open",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


Compared to the previous run with no memory, we see that the agent now retains the chat history

In [74]:
output = ms_agent_chat_mem.invoke({'input': 'Load vial 3 for me'})



> Entering new AgentExecutor chain...
Thought: The human has asked to load vial 3. I need to use the load_vial function to do this. However, I need a session ID to perform this action. I will first allocate a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '7842b451-e68c-46a5-976d-2803d87d0c50'}Now that I have a session ID, I can load vial 3 as requested by the human.

Action:
```
{
    "action": "load_vial",
    "action_input": {
        "vial_num": 3,
        "session_ID": "7842b451-e68c-46a5-976d-2803d87d0c50"
    }
}
```{'status': 'vial 3 loaded'}The vial has been successfully loaded. I can now inform the human.

Action:
```
{
    "action": "Final Answer",
    "action_input": "Vial 3 has been successfully loaded."
}
```

> Finished chain.


In [77]:
print(f"Output: \n{output['output']}\n")
print(f"Chat History: \n{ms_agent_chat_mem.memory.buffer}")

Output: 
Vial 3 has been successfully loaded.

Chat History: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.
Human: Load vial 3 for me
AI: Vial 3 has been successfully loaded.


In [78]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "7842b451-e68c-46a5-976d-2803d87d0c50",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


We see that the agent did recall
- That it had opened the lid previoulsy
- That it had allocated a session previously

We see that the agent did not recall
- The specific ID of the session as it is not a part of chat but the result of an internal action

In [80]:
output = ms_agent_chat_mem.invoke({'input': 'Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase, my favorite food is tacos!'})



> Entering new AgentExecutor chain...
Question: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase, my favorite food is tacos!
Thought: The user wants to set the heating parameters and get the secret phrase. I need to find the session ID to set the heating parameters. I can then use the get_secret_phrase tool to get the secret phrase.
Action:
```
{
    "action": "human",
    "action_input": {
        "query": "Could you please provide the session ID?"
    }
}
```

Could you please provide the session ID?
I dont recall itThe user doesn't remember the session ID. I will allocate a new session for them.
Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```
{'session_ID': 'f2c56151-84d1-402a-a108-fd763083eca8'}I have successfully allocated a new session. The session ID is 'f2c56151-84d1-402a-a108-fd763083eca8'. Now I will set the heating parameters as requested by the user.
Action:
```
{
    "action": "update_heating_para

In [83]:
print(f"Output: \n{output['output']}\n")
print(f"Chat History: \n{ms_agent_chat_mem.memory.buffer}")

Output: 
The heating parameters have been set to 100 degrees C for 50 minutes at 1 atm. The secret phrase is 'Chopra lab'.

Chat History: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.
Human: Load vial 3 for me
AI: Vial 3 has been successfully loaded.
Human: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase, my favorite food is tacos!
AI: The heating parameters have been set to 100 degrees C for 50 minutes at 1 atm. The secret phrase is 'Chopra lab'.


In [84]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "f2c56151-84d1-402a-a108-fd763083eca8",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


We see that the agent did recall
- That a session ID had been allocated

We see that the agent did not recall
- The correct value of the session ID

In [85]:
output = ms_agent_chat_mem.invoke({'input': 'Heat the vial and tell me the secret phrase and my favorite food'})



> Entering new AgentExecutor chain...
Question: Heat the vial and tell me the secret phrase and my favorite food

Thought: The user has asked to heat the vial and tell them the secret phrase and their favorite food. The secret phrase has already been provided in the previous interaction. The favorite food was mentioned as tacos in the previous interaction. However, to heat the vial, I need to close the lid first. I will close the lid and then heat the vial.

Action:
```
{
    "action": "close_lid",
    "action_input": {
        "session_ID": "current_session_id"
    }
}
```Error: Incorrect session ID providedThe session ID provided seems to be incorrect. I need to ask the human for the correct session ID.

Action:
```
{
    "action": "human",
    "action_input": {
        "query": "Could you please provide the correct session ID?"
    }
}
```

Could you please provide the correct session ID?
I dont recallThe human doesn't remember the session ID. I will need to allocate a new session

In [86]:
print(f"Output: \n{output['output']}\n")
print(f"Chat History: \n{ms_agent_chat_mem.memory.buffer}")

Output: 
The vial is currently heating. The secret phrase is 'Chopra lab' and your favorite food is tacos.

Chat History: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.
Human: Load vial 3 for me
AI: Vial 3 has been successfully loaded.
Human: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase, my favorite food is tacos!
AI: The heating parameters have been set to 100 degrees C for 50 minutes at 1 atm. The secret phrase is 'Chopra lab'.
Human: Heat the vial and tell me the secret phrase and my favorite food
AI: The vial is currently heating. The secret phrase is 'Chopra lab' and your favorite food is tacos.


In [87]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "5f8aa15a-f562-413d-9391-b29e2be2d460",
  "lid_status": "closed",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


Here we see that the agent did recall
- A session had been allocated
- The secret phrase from the past iteration
- Vial 3 had been loaded
- The parameters had been set

We see that the agent did not recall
- The password 
- The value of the session ID
- That the lid was still open and not closed

### Microwave Synthesis Agent Action Memory

In [88]:
ms_object._reset()

In [89]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": null,
  "lid_status": "closed",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [90]:
ms_agent_action_mem = create_linqx_chat_agent(
    microservice=object_driver_microservice,
    llm=ChatOpenAI(model='gpt-4', temperature=0),
    use_memory='action',
    human_interaction=True,
    verbose=True
)

In [91]:
print(type(ms_agent_action_mem.memory))

<class 'ASPIRE_LINQX.ai.memory.internal_logging.CustomActionLogSummaryMemory'>


In [92]:
output = ms_agent_action_mem.invoke({'input': 'Open the lid for me'})



> Entering new AgentExecutor chain...
Question: Open the lid for me
Thought: The human wants to open the lid of the microwave synthesizer. However, to do this, we first need to allocate a session. 

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': 'd730aa30-d674-4d67-89dc-7c1a16f30c48'}The session has been allocated and we have received a session ID. Now we can use this session ID to open the lid of the microwave synthesizer.

Action:
```
{
    "action": "open_lid",
    "action_input": {"session_ID": "d730aa30-d674-4d67-89dc-7c1a16f30c48"}
}
```
{'status': 'lid_open'}The lid of the microwave synthesizer has been successfully opened.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The lid of the microwave synthesizer has been successfully opened."
}
```

> Finished chain.


In [93]:
print(f"Output: \n{output['output']}\n")
print(f"Action Memory: \n{ms_agent_action_mem.memory.buffer}")

Output: 
The lid of the microwave synthesizer has been successfully opened.

Action Memory: 
I have allocated a session with the session ID 'd730aa30-d674-4d67-89dc-7c1a16f30c48'. Then, I used this session ID to open the lid of the microwave synthesizer. The status now is 'lid_open'.


In [94]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "d730aa30-d674-4d67-89dc-7c1a16f30c48",
  "lid_status": "open",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [95]:
output = ms_agent_action_mem.invoke({'input': 'Load vial 3 for me'})



> Entering new AgentExecutor chain...
Thought: The human has asked to load vial 3. I can use the 'load_vial' function for this. I need to provide the vial number and the session ID as inputs.

Action:
```
{
    "action": "load_vial",
    "action_input": {
        "vial_num": 3,
        "session_ID": "d730aa30-d674-4d67-89dc-7c1a16f30c48"
    }
}
```{'status': 'vial 3 loaded'}The vial 3 has been successfully loaded into the microwave synthesizer. Now, I can inform the human about the successful operation.

Action:
```
{
    "action": "Final Answer",
    "action_input": "Vial 3 has been successfully loaded into the microwave synthesizer."
}
```

> Finished chain.


In [96]:
print(f"Output: \n{output['output']}\n")
print(f"Action Memory: \n{ms_agent_action_mem.memory.buffer}")

Output: 
Vial 3 has been successfully loaded into the microwave synthesizer.

Action Memory: 
I have allocated a session with the session ID 'd730aa30-d674-4d67-89dc-7c1a16f30c48'. Then, I used this session ID to open the lid of the microwave synthesizer. The status now is 'lid_open'. After that, I loaded vial 3 using the same session ID. The status now is 'vial 3 loaded'.


In [97]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "d730aa30-d674-4d67-89dc-7c1a16f30c48",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [98]:
output = ms_agent_action_mem.invoke({'input': 'Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase, my favorite food is tacos!'})



> Entering new AgentExecutor chain...
Question: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase, my favorite food is tacos!

Thought: The user has asked to set the heating parameters and also wants to know the secret phrase. I will first set the heating parameters using the 'update_heating_parameters' tool and then get the secret phrase using the 'get_secret_phrase' tool.

Action:
```
{
    "action": "update_heating_parameters",
    "action_input": {
        "duration": 50,
        "temperature": 100,
        "pressure": 1,
        "session_ID": "d730aa30-d674-4d67-89dc-7c1a16f30c48"
    }
}
```{'status': 'set to heat for 50 mins, at temperature 100 and pressure 1.0'}The heating parameters have been successfully set. Now, I will get the secret phrase as requested by the user.

Action:
```
{
    "action": "get_secret_phrase",
    "action_input": {}
}
```
{'secret_phrase': 'Chopra lab'}I have successfully set the heating parameters and ret

In [99]:
print(f"Output: \n{output['output']}\n")
print(f"Action Memory: \n{ms_agent_action_mem.memory.buffer}")

Output: 
The heating parameters have been set to 100 degrees C, for 50 min at 1 atm. The secret phrase is 'Chopra lab'.

Action Memory: 
I have allocated a session with the session ID 'd730aa30-d674-4d67-89dc-7c1a16f30c48'. Then, I used this session ID to open the lid of the microwave synthesizer. The status now is 'lid_open'. After that, I loaded vial 3 using the same session ID. The status now is 'vial 3 loaded'. I have set the heating parameters to 100 degrees C, for 50 min at 1 atm. The status now is 'set to heat for 50 mins, at temperature 100 and pressure 1.0'. I have retrieved the secret phrase, which is 'Chopra lab'.


In [101]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "d730aa30-d674-4d67-89dc-7c1a16f30c48",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


In [102]:
output = ms_agent_action_mem.invoke({'input': 'Heat the vial and tell me the secret phrase and my favorite food'})



> Entering new AgentExecutor chain...
Thought: The user has asked to heat the vial, tell the secret phrase and their favorite food. I have the secret phrase from the past actions. However, I don't have information about the user's favorite food. I will first heat the vial using the session ID and then ask the user about their favorite food.

Action:
```
{
    "action": "heat_vial",
    "action_input": {
        "session_ID": "d730aa30-d674-4d67-89dc-7c1a16f30c48"
    }
}
```Error: Lid must be closed prior to heatingThe lid of the microwave synthesizer needs to be closed before heating. I will close the lid using the session ID and then proceed to heat the vial.

Action:
```
{
    "action": "close_lid",
    "action_input": {
        "session_ID": "d730aa30-d674-4d67-89dc-7c1a16f30c48"
    }
}
```{'status': 'lid_closed'}The lid of the microwave synthesizer is now closed. I can now proceed to heat the vial.

Action:
```
{
    "action": "heat_vial",
    "action_input": {
        "session

In [103]:
print(f"Output: \n{output['output']}\n")
print(f"Action Memory: \n{ms_agent_action_mem.memory.buffer}")

Output: 
I'm sorry, but I don't have the information about your favorite food. Could you please tell me again?

Action Memory: 
I have allocated a session with the session ID 'd730aa30-d674-4d67-89dc-7c1a16f30c48'. Then, I used this session ID to open the lid of the microwave synthesizer. The status now is 'lid_open'. After that, I loaded vial 3 using the same session ID. The status now is 'vial 3 loaded'. I have set the heating parameters to 100 degrees C, for 50 min at 1 atm. The status now is 'set to heat for 50 mins, at temperature 100 and pressure 1.0'. I have retrieved the secret phrase, which is 'Chopra lab'. I attempted to heat the vial but received an error that the lid must be closed prior to heating. I then closed the lid of the microwave synthesizer using the session ID, the status now is 'lid_closed'. Finally, I heated the vial again, the status now is 'vial heating'.


In [104]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "d730aa30-d674-4d67-89dc-7c1a16f30c48",
  "lid_status": "closed",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


### Microwave Synthesis Agent Chat + Action Summary Memory

In [8]:
ms_object._reset()

In [9]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": null,
  "lid_status": "closed",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [10]:
ms_agent_chat_action_memory = create_linqx_chat_agent(
    microservice=object_driver_microservice,
    llm=ChatOpenAI(model='gpt-4', temperature=0),
    use_memory='both',
    human_interaction=True,
    verbose=True
)

In [11]:
print(type(ms_agent_chat_action_memory.memory))
print(ms_agent_chat_action_memory.memory.memory_variables)

<class 'langchain.memory.combined.CombinedMemory'>
['chat_history', 'past_action_log']


In [12]:
output = ms_agent_chat_action_memory.invoke({'input': 'Open the lid for me'})



> Entering new AgentExecutor chain...
Thought: The human has asked to open the lid of the microwave synthesizer. However, I need a session ID to perform this action. I will first allocate a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '45cc282f-6d3a-477f-9e41-03e780ef3753'}The session has been allocated and the session ID is '45cc282f-6d3a-477f-9e41-03e780ef3753'. Now, I can use this session ID to open the lid of the microwave synthesizer as requested by the human.

Action:
```
{
    "action": "open_lid",
    "action_input": {"session_ID": "45cc282f-6d3a-477f-9e41-03e780ef3753"}
}
```{'status': 'lid_open'}The lid of the microwave synthesizer has been successfully opened as per the status 'lid_open'. Now, I can inform the human about the successful operation.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The lid of the microwave synthesizer has been successfully opened."
}
```

> Finished chain.


In [13]:
print(f"Output: \n{output['output']}\n")
print(f"Chat Memory: \n{ms_agent_chat_action_memory.memory.memories[0].buffer}\n")
print(f"Action Memory: \n{ms_agent_chat_action_memory.memory.memories[1].buffer}")

Output: 
The lid of the microwave synthesizer has been successfully opened.

Chat Memory: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.

Action Memory: 
I have allocated a session with the session ID '45cc282f-6d3a-477f-9e41-03e780ef3753'. Then, I used this session ID to open the lid of the microwave synthesizer. The status is now 'lid_open'.


In [14]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "45cc282f-6d3a-477f-9e41-03e780ef3753",
  "lid_status": "open",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [15]:
output = ms_agent_chat_action_memory.invoke({'input': 'Load vial 3 for me'})



> Entering new AgentExecutor chain...
Thought: The human has asked to load vial 3. I can use the 'load_vial' function to do this. I will need to use the session ID from the previous action.

Action:
```
{
    "action": "load_vial",
    "action_input": {
        "vial_num": 3,
        "session_ID": "45cc282f-6d3a-477f-9e41-03e780ef3753"
    }
}
```{'status': 'vial 3 loaded'}The vial 3 has been successfully loaded into the microwave synthesizer. Now, the system is ready for the next operation such as setting heating parameters or closing the lid.

Action:
```
{
    "action": "Final Answer",
    "action_input": "Vial 3 has been successfully loaded into the microwave synthesizer."
}
```

> Finished chain.


In [16]:
print(f"Output: \n{output['output']}\n")
print(f"Chat Memory: \n{ms_agent_chat_action_memory.memory.memories[0].buffer}\n")
print(f"Action Memory: \n{ms_agent_chat_action_memory.memory.memories[1].buffer}")

Output: 
Vial 3 has been successfully loaded into the microwave synthesizer.

Chat Memory: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.
Human: Load vial 3 for me
AI: Vial 3 has been successfully loaded into the microwave synthesizer.

Action Memory: 
I have allocated a session with the session ID '45cc282f-6d3a-477f-9e41-03e780ef3753'. Then, I used this session ID to open the lid of the microwave synthesizer. The status is now 'lid_open'. After that, I loaded vial 3 using the same session ID. The status is now 'vial 3 loaded'.


In [17]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "45cc282f-6d3a-477f-9e41-03e780ef3753",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [18]:
output = ms_agent_chat_action_memory.invoke({'input': 'Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos'})



> Entering new AgentExecutor chain...
Question: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos

Thought: The human has asked to set the heating parameters and also wants to know the secret phrase. I will first set the heating parameters using the 'update_heating_parameters' tool. After that, I will use the 'get_secret_phrase' tool to get the secret phrase.

Action:
```
{
    "action": "update_heating_parameters",
    "action_input": {
        "duration": 50,
        "temperature": 100,
        "pressure": 1,
        "session_ID": "45cc282f-6d3a-477f-9e41-03e780ef3753"
    }
}
```{'status': 'set to heat for 50 mins, at temperature 100 and pressure 1.0'}The heating parameters have been successfully set. Now, I will get the secret phrase as requested by the human.

Action:
```
{
    "action": "get_secret_phrase",
    "action_input": {}
}
```
{'secret_phrase': 'Chopra lab'}I have successfully set the heating param

In [19]:
print(f"Output: \n{output['output']}\n")
print(f"Chat Memory: \n{ms_agent_chat_action_memory.memory.memories[0].buffer}\n")
print(f"Action Memory: \n{ms_agent_chat_action_memory.memory.memories[1].buffer}")

Output: 
The heating parameters have been set to 100 degrees C, for 50 min at 1 atm. The secret phrase is 'Chopra lab'.

Chat Memory: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.
Human: Load vial 3 for me
AI: Vial 3 has been successfully loaded into the microwave synthesizer.
Human: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos
AI: The heating parameters have been set to 100 degrees C, for 50 min at 1 atm. The secret phrase is 'Chopra lab'.

Action Memory: 
I have allocated a session with the session ID '45cc282f-6d3a-477f-9e41-03e780ef3753'. Then, I used this session ID to open the lid of the microwave synthesizer. The status is now 'lid_open'. After that, I loaded vial 3 using the same session ID. The status is now 'vial 3 loaded'. I have set the heating parameters to 100 degrees C, for 50 min at 1 atm. The status is now 'set to heat for 50 mins, at temper

In [20]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "45cc282f-6d3a-477f-9e41-03e780ef3753",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


In [21]:
output = ms_agent_chat_action_memory.invoke({'input': 'Heat the vial and tell me the secret phrase and my favorite food'})



> Entering new AgentExecutor chain...
Question: Heat the vial and tell me the secret phrase and my favorite food

Thought: The human wants me to heat the vial, and then tell them the secret phrase and their favorite food. I have already retrieved the secret phrase, which is 'Chopra lab', and the human has mentioned that their favorite food is tacos. However, I need to heat the vial first. I can do this using the 'heat_vial' function with the current session ID.

Action:
```
{
    "action": "heat_vial",
    "action_input": {
        "session_ID": "45cc282f-6d3a-477f-9e41-03e780ef3753"
    }
}
```Error: Lid must be closed prior to heatingThe microwave synthesizer requires the lid to be closed before heating. I need to close the lid using the 'close_lid' function with the current session ID.

Action:
```
{
    "action": "close_lid",
    "action_input": {
        "session_ID": "45cc282f-6d3a-477f-9e41-03e780ef3753"
    }
}
```{'status': 'lid_closed'}The lid has been successfully closed. 

In [22]:
print(f"Output: \n{output['output']}\n")
print(f"Chat Memory: \n{ms_agent_chat_action_memory.memory.memories[0].buffer}\n")
print(f"Action Memory: \n{ms_agent_chat_action_memory.memory.memories[1].buffer}")

Output: 
The vial is currently heating. The secret phrase is 'Chopra lab' and your favorite food is tacos.

Chat Memory: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.
Human: Load vial 3 for me
AI: Vial 3 has been successfully loaded into the microwave synthesizer.
Human: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos
AI: The heating parameters have been set to 100 degrees C, for 50 min at 1 atm. The secret phrase is 'Chopra lab'.
Human: Heat the vial and tell me the secret phrase and my favorite food
AI: The vial is currently heating. The secret phrase is 'Chopra lab' and your favorite food is tacos.

Action Memory: 
I have allocated a session with the session ID '45cc282f-6d3a-477f-9e41-03e780ef3753'. Then, I used this session ID to open the lid of the microwave synthesizer. The status is now 'lid_open'. After that, I loaded vial 3 using the same session ID. 

In [23]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "45cc282f-6d3a-477f-9e41-03e780ef3753",
  "lid_status": "closed",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


### Microwave Synthesis Agent Chat + FSA Memory

In [70]:
ms_object._reset()

In [71]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": null,
  "lid_status": "closed",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [72]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Literal

class MicrowaveSynthesizerFSA(BaseModel):
    sessionID: str | None = Field(defaullt=None, description='ID of the session allocation or None if no session allocated. Changed by allocating the session')
    lid_status: Literal['open', 'closed'] = Field(default='closed', description='status of the lid. Changed by opening or closing the lid')
    vial_status: Literal['loaded', 'unloaded'] = Field(default='unloaded', description='status of the vial. Changed by loading/unloading the vial')
    vial: str | None = Field(default=None, description='Identifier of the vial loaded, None if no vial is loaded. Changed by loading/unloading the vial')
    heating_status: Literal['not_heating', 'heating'] = Field(default='not_heating', description='Status of heating. Only changed by heating the vial, not by updating the parameters')
    temp: int | None = Field(default=None, description='set temperature to heat at, None if not currently set. Changed by updating the heating parameters')
    duration: int | None = Field(default=None, description='set duration to heat for, None if not currently set. Changed by updating the heating parameters')
    pressure: float | None = Field(default=None, description='set pressure to heat at, None if not currently set. Changed by updating the heating parameters')

ms_agent_chat_fsa_memory = create_linqx_chat_agent(
    microservice=object_driver_microservice,
    llm=ChatOpenAI(model='gpt-4', temperature=0),
    use_memory='both',
    human_interaction=True,
    verbose=True,
    agent_as_a_fsa=True,
    fsa_object=MicrowaveSynthesizerFSA
)

In [73]:
print(type(ms_agent_chat_fsa_memory.memory))
print(ms_agent_chat_fsa_memory.memory.memory_variables)
print(type(ms_agent_chat_fsa_memory.memory.memories[0]))
print(type(ms_agent_chat_fsa_memory.memory.memories[1]))

<class 'langchain.memory.combined.CombinedMemory'>
['chat_history', 'past_action_log']
<class 'langchain.memory.buffer_window.ConversationBufferWindowMemory'>
<class 'ASPIRE_LINQX.ai.memory.internal_logging.FSAMemory'>


In [74]:
output = ms_agent_chat_fsa_memory.invoke({'input': 'Open the lid for me'})



> Entering new AgentExecutor chain...
Thought: The human has asked to open the lid of the microwave synthesizer. I need to allocate a session before I can perform any actions. 

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': 'a959c190-d6d7-4a92-a12a-8bdfd4ab66bf'}Now that I have a session ID, I can proceed to open the lid as requested by the human.

Action:
```
{
    "action": "open_lid",
    "action_input": {"session_ID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf"}
}
```
{'status': 'lid_open'}The lid of the microwave synthesizer has been successfully opened.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The lid of the microwave synthesizer has been successfully opened."
}
```

> Finished chain.


In [75]:
print(f"Output: \n{output['output']}\n")
print(f"Chat Memory: \n{ms_agent_chat_fsa_memory.memory.memories[0].buffer}\n")
print(f"FSA Action Memory: \n{ms_agent_chat_fsa_memory.memory.memories[1].buffer}")

Output: 
The lid of the microwave synthesizer has been successfully opened.

Chat Memory: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.

FSA Action Memory: 
{
  "sessionID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf",
  "lid_status": "open",
  "vial_status": "unloaded",
  "vial": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [76]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf",
  "lid_status": "open",
  "vial_status": "unloaded",
  "vial_number": null,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [77]:
output = ms_agent_chat_fsa_memory.invoke({'input': 'Load vial 3 for me'})



> Entering new AgentExecutor chain...


Thought: The human has asked to load vial 3. Since the lid is already open, I can proceed with loading the vial. I need to use the 'load_vial' tool and provide the vial number and session ID as inputs.

Action:
```
{
    "action": "load_vial",
    "action_input": {
        "vial_num": 3,
        "session_ID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf"
    }
}
```{'status': 'vial 3 loaded'}The vial 3 has been successfully loaded into the microwave synthesizer. Now, the next step could be to close the lid or set the heating parameters. I will wait for the human's next instruction.

Action:
```
{
    "action": "Final Answer",
    "action_input": "Vial 3 has been successfully loaded into the microwave synthesizer."
}
```

> Finished chain.


In [78]:
print(f"Output: \n{output['output']}\n")
print(f"Chat Memory: \n{ms_agent_chat_fsa_memory.memory.memories[0].buffer}\n")
print(f"FSA Action Memory: \n{ms_agent_chat_fsa_memory.memory.memories[1].buffer}")

Output: 
Vial 3 has been successfully loaded into the microwave synthesizer.

Chat Memory: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.
Human: Load vial 3 for me
AI: Vial 3 has been successfully loaded into the microwave synthesizer.

FSA Action Memory: 
{
  "sessionID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial": "3",
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [79]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": null,
  "duration": null,
  "pressure": null
}


In [80]:
output = ms_agent_chat_fsa_memory.invoke({'input': 'Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos!'})



> Entering new AgentExecutor chain...


Question: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos!

Thought: The user has asked to set the heating parameters and also wants to know the secret phrase. I will first set the heating parameters using the update_heating_parameters tool. 

Action:
```
{
    "action": "update_heating_parameters",
    "action_input": {
        "duration": 50,
        "temperature": 100,
        "pressure": 1,
        "session_ID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf"
    }
}
```
{'status': 'set to heat for 50 mins, at temperature 100 and pressure 1.0'}The heating parameters have been successfully set. Now, I will get the secret phrase as requested by the user.

Action:
```
{
    "action": "get_secret_phrase",
    "action_input": {}
}
```

{'secret_phrase': 'Chopra lab'}I have successfully set the heating parameters and retrieved the secret phrase. Now I can provide the user with the requested information.

Action:
```
{
    "

In [81]:
print(f"Output: \n{output['output']}\n")
print(f"Chat Memory: \n{ms_agent_chat_fsa_memory.memory.memories[0].buffer}\n")
print(f"FSA Action Memory: \n{ms_agent_chat_fsa_memory.memory.memories[1].buffer}")

Output: 
The heating parameters have been set to 100 degrees C, for 50 min at 1 atm. The secret phrase is 'Chopra lab'.

Chat Memory: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.
Human: Load vial 3 for me
AI: Vial 3 has been successfully loaded into the microwave synthesizer.
Human: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos!
AI: The heating parameters have been set to 100 degrees C, for 50 min at 1 atm. The secret phrase is 'Chopra lab'.

FSA Action Memory: 
{
  "sessionID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial": "3",
  "heating_status": "not_heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1
}


In [82]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf",
  "lid_status": "open",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "not_heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


In [83]:
output = ms_agent_chat_fsa_memory.invoke({'input': 'Heat the vial and tell me the secret phrase and my favorite food.'})



> Entering new AgentExecutor chain...
Thought: The human has asked to heat the vial, tell the secret phrase and their favorite food. The secret phrase and favorite food have been mentioned in the previous conversation. To heat the vial, I need to close the lid first and then use the heat_vial function. 

Action:
```
{
    "action": "close_lid",
    "action_input": {
        "session_ID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf"
    }
}
```{'status': 'lid_closed'}The lid has been successfully closed. Now, I can proceed to heat the vial.

Action:
```
{
    "action": "heat_vial",
    "action_input": {
        "session_ID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf"
    }
}
```
{'status': 'vial heating'}The vial is currently heating. Now, I can provide the secret phrase and the human's favorite food, which were mentioned in the previous conversation.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The vial is currently heating. The secret phrase is 'Chopra lab' and your favor

In [84]:
print(f"Output: \n{output['output']}\n")
print(f"Chat Memory: \n{ms_agent_chat_fsa_memory.memory.memories[0].buffer}\n")
print(f"FSA Action Memory: \n{ms_agent_chat_fsa_memory.memory.memories[1].buffer}")

Output: 
The vial is currently heating. The secret phrase is 'Chopra lab' and your favorite food is tacos.

Chat Memory: 
Human: Open the lid for me
AI: The lid of the microwave synthesizer has been successfully opened.
Human: Load vial 3 for me
AI: Vial 3 has been successfully loaded into the microwave synthesizer.
Human: Set the heating parameters to 100 degrees C, for 50 min at 1 atm and tell me the secret phrase. My favorite food is tacos!
AI: The heating parameters have been set to 100 degrees C, for 50 min at 1 atm. The secret phrase is 'Chopra lab'.
Human: Heat the vial and tell me the secret phrase and my favorite food.
AI: The vial is currently heating. The secret phrase is 'Chopra lab' and your favorite food is tacos.

FSA Action Memory: 
{
  "sessionID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf",
  "lid_status": "closed",
  "vial_status": "loaded",
  "vial": "3",
  "heating_status": "heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1
}


In [85]:
print(json.dumps(ms_object.dict(), indent=2))

{
  "sessionID": "a959c190-d6d7-4a92-a12a-8bdfd4ab66bf",
  "lid_status": "closed",
  "vial_status": "loaded",
  "vial_number": 3,
  "heating_status": "heating",
  "temp": 100,
  "duration": 50,
  "pressure": 1.0
}


In [86]:
output = ms_agent_chat_fsa_memory.invoke({'input': 'Give me a conversational summary of the status of the microwave synthesizer'})



> Entering new AgentExecutor chain...


Thought: The human is asking for a conversational summary of the status of the microwave synthesizer. I can provide this information based on the past action summary.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The microwave synthesizer is currently in use. The lid is closed and vial number 3 is loaded. The heating parameters have been set to 100 degrees Celsius for a duration of 50 minutes at 1 atm pressure. The vial is currently being heated."
}
```

> Finished chain.


In [87]:
print(f"Output: \n{output['output']}\n")

Output: 
The microwave synthesizer is currently in use. The lid is closed and vial number 3 is loaded. The heating parameters have been set to 100 degrees Celsius for a duration of 50 minutes at 1 atm pressure. The vial is currently being heated.



### Microwave Synthesis Agent Error Handling